# <font color='blue'>ALPAR - Governo Digital - Processo Inteligente</font>

# <font color='red'>recomendação de ações - gravando em MySQL</font>

In [1]:
# !pip install mysql-connector-python

In [2]:
import pandas as pd
import numpy as np
import copy
import csv
import codecs
import time
from os.path import expanduser

print("pandas versão", pd.__version__)
print("numpy versão", np.__version__)
print("csv versão", csv.__version__)

pd.options.display.max_rows = 2000

pandas versão 1.1.5
numpy versão 1.19.4
csv versão 1.0


# <font color='black'>trata o dataset total</font>

In [3]:
Simulacao = True                            # alterna True ou False para tratar de dados reias ou simulados

# caminho dos arquivos...
home = expanduser("~")
caminho_in = home + ('/Downloads/GD/' if Simulacao else '/Downloads/GDR/')
arq_tasks = caminho_in + ('n_tasks.csv' if Simulacao else 'tasks.csv')

df_tasks = pd.read_csv(arq_tasks, sep = ',', dtype = str)
print("linhas = %s , colunas = %s " % df_tasks.shape)

# df_tasks

linhas = 3756079 , colunas = 23 


In [4]:
# exclui colunas não utilizáveis
lst_del = ['Entidade', 'Usuário', 'Grupo', 
           'Motivo de cancelamento', 'Notificação',
           'Comentário', 'Apoio', 'Data e Hora de atendimento',
           'Status externo', 'Data de agendamento', 'Agendamento',
           'Categoria', 'Grupo responsável', 'Prazo (em segundos)']
df_tasks.drop(lst_del, axis = 1, inplace = True)
lst_nomes = ['entidade', 'servico', 'protocolo', 'dt_conclusao', 'dt_criacao',
             'acao', 'encaminhado_para', 'encerrado', 'cancelado']
df_tasks.columns = lst_nomes

In [5]:
# transforma datas do tipo string para tipo data
df_tasks['dt_conclusao']= pd.to_datetime(df_tasks['dt_conclusao'])
df_tasks['dt_criacao']= pd.to_datetime(df_tasks['dt_criacao'])

In [6]:
# verifica a situação de cada protocolo
df = df_tasks[['protocolo', 'encerrado', 'cancelado']].copy()

df.loc[df['encerrado'] == 'false', 'encerrado'] = 0
df.loc[df['encerrado'] == 'true', 'encerrado'] = 1
df.loc[df['cancelado'] == 'false', 'cancelado'] = 0
df.loc[df['cancelado'] == 'true', 'cancelado'] = 1

df['encerrado'] = pd.to_numeric(df['encerrado'])
df['cancelado'] = pd.to_numeric(df['cancelado'])

# df

In [7]:
# agrupa por protocolo, obtem o máximo de encerrado e cancelado
df = df.groupby('protocolo').agg({'encerrado': 'max', 'cancelado': 'max'})

In [8]:
# filtra os protocolos cancelados
df.drop(df[df['cancelado'] == 1].index, inplace=True)

# exclui a coluna de protocolos cancelados
df.drop('cancelado', axis=1, inplace=True)

# cria um dataframe de processos encerrados
df_ence = df[(df['encerrado'] == 1)].copy()

# cria um dataframe de processos em andamento
df_anda = df[(df['encerrado'] == 0)].copy()

# exclui a coluna encerrado dos 2 dataframes
df_ence.drop('encerrado', axis=1, inplace=True)
df_anda.drop('encerrado', axis=1, inplace=True)

In [9]:
print(df_ence.shape)
print(df_anda.shape)

(346059, 0)
(49124, 0)


In [10]:
# cria o dataframe de encerrados e dataframe de em andamento
lst_ence = df_ence.index.tolist()
lst_anda = df_anda.index.tolist()

# <font color='black'>cria e trata o dataset de protocolos encerrados</font>

In [11]:
# cria dataset de tasks encerradas
df_ence = df_tasks[df_tasks['protocolo'].isin(lst_ence)].copy()

In [12]:
# verifica dados nulos
# df_ence.isnull().sum()

In [13]:
# coloca a string <vazio> nos valores NULL
lst_mod = list(df_ence[df_ence['encaminhado_para'].isnull()].index)
df_ence.loc[lst_mod, 'encaminhado_para'] = '<Vazio>'

In [14]:
# ordena por entidade, serviço, protocolo, dt_criação
df_ence.sort_values(by = ['entidade', 'servico', 'protocolo', 'dt_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_ence.reset_index(drop = True, inplace = True)

In [15]:
# calcula a quantidade de dias da ação e exclui as datas de criação e de conclusão
df_ence['dias'] = (df_ence['dt_conclusao'] - df_ence['dt_criacao']) / np.timedelta64(1, 'D')
df_ence.drop(['dt_conclusao', 'dt_criacao'], axis=1, inplace=True)

In [16]:
# exclui colunas não utilizáveis
df_ence.drop(['encerrado', 'cancelado'], axis=1, inplace=True)

# df_ence

In [17]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['protocolo', 'entidade', 'servico'])
      .agg(
        lst_acao=('acao', lambda x: tuple(x)),
        lst_encaminhado=('encaminhado_para', lambda x: tuple(x)),
        sum_dias=('dias', sum)).reset_index())

In [18]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['entidade', 'servico', 'lst_acao', 'lst_encaminhado'])
      .agg(
        media_dias=('sum_dias', 'mean')).reset_index())

# <font color='black'>cria o dataset de protocolos em andamento</font>

In [19]:
# cria dataset de tasks em andamento
df_anda = df_tasks[df_tasks['protocolo'].isin(lst_anda)].copy()

In [20]:
# exclui colunas não utilizáveis
df_anda.drop(['encerrado', 'cancelado', 'dt_conclusao'], axis=1, inplace=True)

# exclui as linhas com a coluna ação NULL
df_anda = df_anda[~df_anda['acao'].isnull()].copy()

# df_anda

In [21]:
# ordena por entidade, serviço, protocolo, dt_criação
df_anda.sort_values(by = ['entidade', 'servico', 'protocolo', 'dt_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_anda.reset_index(drop = True, inplace = True)

In [22]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_anda = (df_anda.groupby(['protocolo', 'entidade', 'servico'])
      .agg(lst_acao=('acao', lambda x: tuple(x)))
      .reset_index()).copy()
# df_anda

# <font color='black'>TESTES</font>

# <font color='black'>cria um arquivo com recomendações</font>

In [26]:
start = time.process_time()
tot = 0

# analisa cada protocolo em andamento
# procura a sequência do protocolo em andamento para sugerir o restante da sequência

# zera a lista de recomendações
lst_reco = []
const_max_tempo = 999999999.9
tot_anda = range(len(df_anda))
print('total de protocolos a processar =', tot_anda)

# para cada protocolo em andamento procura uma coleção de sequências dos protocolos encerrados
for i in tot_anda:
    n_acoes = len(df_anda.loc[i, 'lst_acao'])
    df_aux = df_ence[(df_ence['entidade'] == df_anda.loc[i, 'entidade']) & 
                     (df_ence['servico'] == df_anda.loc[i, 'servico']) &
                     (df_ence['lst_acao'].apply(lambda x: x[:n_acoes]) == df_anda.loc[i, 'lst_acao'])]
    df_aux.reset_index(inplace = True)
    
    # zera a variável que vai achar o registro "entidade/serviço" com o menor tempo de conclusão
    # para quando for achado mais de 1 possibilidade
    min_tempo = const_max_tempo
    dt = {'sequencia': [], 'encaminhamento': []}

    qtd_casos = len(df_aux)
    tot += qtd_casos
    
    for k in range(qtd_casos):
        valor = np.float64(df_aux.loc[k, 'media_dias'])
        if valor < min_tempo or min_tempo == None:
            min_tempo = valor
            dt = {'sequencia': list(df_aux.loc[k, 'lst_acao']), 'encaminhamento': list(df_aux.loc[k, 'lst_encaminhado'])}
            
    indice = len(list(df_anda.loc[i, 'lst_acao'])) if dt else -1
    if qtd_casos == 1:
        tipo_recomendacao = 2
    elif qtd_casos > 1:
        tipo_recomendacao = 3
    else:
        tipo_recomendacao = 1
        
    it = {'protocolo': df_anda.loc[i, 'protocolo'],
            'seq_atual': list(df_anda.loc[i, 'lst_acao']),
            'recomendacao': dt,
            'tempo_medio': None if min_tempo == const_max_tempo > 5 else min_tempo,
            'reco_acao_idx': indice,
            'qtd_casos' : qtd_casos,
            'tipo_recomendacao' : tipo_recomendacao}
    lst_reco.append(it)
    if i % 100 == 0:
        print(i, tot)

print(time.process_time() - start)

total de protocolos a processar = range(0, 49124)
0 0
100 2576
200 4825
300 8687
400 11574
500 14056
600 19141
700 22772
800 26699
900 31256
1000 36183
1100 41446
1200 46026
1300 51715
1400 56930
1500 63701
1600 67576
1700 73672
1800 78060
1900 82994
2000 90382
2100 94752
2200 98545
2300 103813
2400 109817
2500 111725
2600 116544
2700 121048
2800 124514
2900 128471
3000 134658
3100 139741
3200 143653
3300 147297
3400 149608
3500 152883
3600 157472
3700 161492
3800 163223
3900 166026
4000 167596
4100 169852
4200 172632
4300 175277
4400 177726
4500 179134
4600 180963
4700 182691
4800 186220
4900 191281
5000 196102
5100 199045
5200 202138
5300 206306
5400 210350
5500 213099
5600 214887
5700 221076
5800 223384
5900 226769
6000 230398
6100 232926
6200 239690
6300 242802
6400 246090
6500 248798
6600 251521
6700 255985
6800 259883
6900 264272
7000 268268
7100 271377
7200 274391
7300 277728
7400 281171
7500 285150
7600 289795
7700 293457
7800 296987
7900 302938
8000 307027
8100 310041
8200 315

In [27]:
# prepara uma lista de tuplas para gravação da tabela "tb_protocolo" no MySQL
lst_protocolo = []
lst_recomenda = []
for k in lst_reco:
    prx_acao = None
    prx_encaminhamento = None
    
    n_acoes_ence = len(k['recomendacao']['sequencia'])
    for j in range(n_acoes_ence):
        if j == k['reco_acao_idx']:
            proximo = 1
            prx_acao = k['recomendacao']['sequencia'][j]
            prx_encaminhamento = k['recomendacao']['encaminhamento'][j]
        else:
            proximo = 0
        
        lst_recomenda.append((j, 
                              k['protocolo'], 
                              k['recomendacao']['sequencia'][j], 
                              k['recomendacao']['encaminhamento'][j], 
                              proximo
                             )
        )

    lst_protocolo.append((k['protocolo'], 
                          k['tempo_medio'], 
                          prx_acao, 
                          prx_encaminhamento,
                          k['qtd_casos'],
                          k['tipo_recomendacao']
                         )
    )

In [28]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='bd_reco',
                                         user='gd',
                                         password='Alpar@123')
    
    insert_protocolo = """INSERT INTO tb_protocolo 
                        (pk_protocolo, tp_medio, nm_prox_acao, nm_prox_enc, nu_qtd_casos, cd_tipo_reco) 
                        VALUES (%s, %s, %s, %s, %s, %s)"""
    insert_recomenda = """INSERT INTO tb_recomenda 
                        (nu_seq, fk_protocolo, nm_acao, nm_encaminhado, fg_proximo) 
                        VALUES (%s, %s, %s, %s, %s)"""


    cursor = connection.cursor()

    cursor.execute("TRUNCATE TABLE bd_reco.tb_recomenda")

    cursor.execute("SET SQL_SAFE_UPDATES = 0")
    cursor.execute("DELETE FROM bd_reco.tb_protocolo")
    cursor.execute("SET SQL_SAFE_UPDATES = 1")

    cursor.executemany (insert_protocolo, lst_protocolo)
    cursor.executemany (insert_recomenda, lst_recomenda)

    connection.commit()

    print(cursor.rowcount, "Record inserted successfully into tb_protocolo table")

except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

87853 Record inserted successfully into tb_protocolo table
MySQL connection is closed
